In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import copy

In [2]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [18]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
	for person in keypoints_with_scores:
		y, x, c = frame.shape
		shaped = np.squeeze(np.multiply(person, [y,x,1]))

		for i, kp in enumerate(shaped):
			if i == 5 or i == 6:
				ky, kx, kp_conf = kp
				if kp_conf > confidence_threshold:
					cv.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)
					cv.putText(frame, text=f"{i}", org=(int(kx), int(ky)),
							fontFace=cv.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0, 0, 0), thickness=1, lineType=cv.LINE_AA)

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [19]:
OriginalImage = cv.imread('E:\GithubProjects\HelmetDetection\TestImage.jpg')

while True:
	ShowImage = copy.deepcopy(OriginalImage)
	Height, Width = ShowImage.shape[:2]
	# Width : Height = 400 : ?
	ShowImage = cv.resize(ShowImage, (400, int(400 * Height / Width)))
	Height, Width = ShowImage.shape[:2]


	# Resize image
	img = ShowImage.copy()
	img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
	input_img = tf.cast(img, dtype=tf.int32)

	# Detection section
	results = movenet(input_img)
	keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

	# Render keypoints 
	loop_through_people(ShowImage, keypoints_with_scores, EDGES, 0.1)

	cv.imshow('Movenet Multipose', ShowImage)

	if cv.waitKey(1) == ord('q'):
		break
cv.destroyAllWindows()